In [1]:
import numpy as np
import pandas as pd
import re
import cv2
import os
import shutil 
import glob
import random
import sys
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
my_local_drive='/content/gdrive/My Drive/TER/Pred/'
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive

%pwd

/content/gdrive/My Drive/TER/Pred


'/content/gdrive/My Drive/TER/Pred'

In [4]:
#!rm -rf imgs
#!rm -rf imgs_cropped
!unzip "/content/gdrive/MyDrive/TER/Pred/imgs.zip"
#!rm imgs.zip

Archive:  /content/gdrive/MyDrive/TER/Pred/imgs.zip
replace test/w_3188.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
!mkdir imgs_cropped

In [5]:
# Load Yolo
net = cv2.dnn.readNet("yolov3_training_4000.weights", "yolov3_testing.cfg")

# Name custom object
classes = ["Whale"]

# Images path
images_path = glob.glob("test/*.jpg")
layer_names = net.getLayerNames()
output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
colors = np.random.uniform(0, 255, size=(len(classes), 3))

for img_path in images_path:
  # Loading image
  img = cv2.imread(img_path)
  img = cv2.resize(img, None, fx=0.4, fy=0.4)
  height, width, channels = img.shape

  # Detecting objects
  blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=True)

  net.setInput(blob)
  outs = net.forward(output_layers)

  # Showing informations on the screen
  class_ids = []
  confidences = []
  boxes = []
  for out in outs:
      for detection in out:
          scores = detection[5:]
          class_id = np.argmax(scores)
          confidence = scores[class_id]
          if confidence > 0.3:
              # Object detected
              center_x = int(detection[0] * width)
              center_y = int(detection[1] * height)
              w = int(detection[2] * width)
              h = int(detection[3] * height)

              # Rectangle coordinates
              x = int(center_x - w / 2)
              y = int(center_y - h / 2)

              boxes.append([x, y, w, h])
              confidences.append(float(confidence))
              class_ids.append(class_id)

  indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
  font = cv2.FONT_HERSHEY_PLAIN
  for i in range(len(boxes)):
      if i in indexes:
          x, y, w, h = boxes[i]
          label = str(classes[class_ids[i]])
          color = colors[class_ids[i]]
          cropped_img = img[y:y+h,x:x+w]
          name = re.findall('test\/(.*)',img_path).pop()
          print(x, y, w, h)
          if x<0 or y<0 or y+h > img.shape[0] or x+w > img.shape[1]:
            continue    
          cv2.imwrite("imgs_cropped/"+name ,cropped_img) 
                  

581 77 769 394
679 381 479 131
506 368 1531 482
876 870 576 525
455 221 370 544
-5 270 771 278
369 194 352 602
498 265 187 250
475 346 316 418
228 127 904 357
468 354 644 341
1129 386 633 558
124 51 618 491
414 234 400 518
-19 230 702 485


In [ ]:
df = pd.read_csv("train.csv")
imgs_labelized_name = df['Image'].values.astype("str")
#imgs_labelized_name = ["imgs/" + s for s in imgs_labelized_name]


In [ ]:
os.makedirs("imgs_label")
os.makedirs("imgs_no_label")

In [ ]:
for name in arr:
  shutil.move("imgs/" + name, "imgs_label/"+ name) 